# Group Project

As a Data Scientist, you are tasked to help these users find the most interesting articles
according to their preferred topics. You have a ***training dataset containing about 9500 news
articles, each assigned to one of the above topics***. In addition, (as in real life situation) the
dataset contains about ***48% of irrelevant articles*** (marked as IRRELEVANT) that do not
belong to any of the topics; hence the users are not interested in them. The distribution of
articles over topics is not uniform. There are some topics with large number of articles, and
some with very small number.

One day, 500 new articles have been published. This is your test set that has similar article
distribution over topics to the training set. ***Your task is to suggest up to 10 of the most relevant
articles from this set of 500 to each user***. The number of suggestions is limited to 10, because,
presumably, the users do not want to read more suggestions. It is possible, however, that some
topics within this test set have less than 10 articles. You also do not want to suggest 10 articles
if they are unlikely to be relevant, because you are concerned that the users may get
discouraged and stop using your application altogether. Therefore you need to take a balanced
approach, paying attention to not suggesting too many articles for rare topics.

### Import Library

In [1]:
# Load libraries
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import collections


### Pre-processing

In [2]:
# Import text data
raw_training = pd.read_csv("training.csv")
raw_testing = pd.read_csv("test.csv")

# Create bag of words
count = CountVectorizer()
bag_of_words = count.fit_transform(raw_training["article_words"])

# Create feature matrix
X = bag_of_words

# Create bag of words
y = raw_training["topic"]

#######################Resampling Dataset#######################


# Reducing the effect imbalnced by deleting some irrelevant class
# "Irrelevant" classe has 4734 samples in the training data, try to reduce it into 2000
irrelevant = raw_training[raw_training["topic"] == "IRRELEVANT"]
remove_n = 2734
drop_indices = np.random.choice(irrelevant.index, remove_n, replace=False)
irrelevant = irrelevant.drop(drop_indices)
zx
reduce_training =  pd.concat([raw_training[raw_training["topic"] != "IRRELEVANT"], irrelevant],ignore_index=True)
reduce_bag_of_words = count.fit_transform(reduce_training["article_words"])
R_X = reduce_bag_of_words
R_y = reduce_training["topic"]

# Icreasing the minor classes
# Increasing 
topic_class = raw_training[raw_training["topic"] != "IRRELEVANT"]
increase_training = pd.concat([topic_class, topic_class, topic_class, raw_training[raw_training["topic"] == "IRRELEVANT"]], ignore_index=True)
increase_bag_of_words = count.fit_transform(increase_training["article_words"])
I_X = increase_bag_of_words
I_y = increase_training["topic"]



In [3]:
y

0       FOREX MARKETS
1       MONEY MARKETS
2              SPORTS
3       FOREX MARKETS
4          IRRELEVANT
            ...      
9495          DEFENCE
9496       IRRELEVANT
9497    FOREX MARKETS
9498       IRRELEVANT
9499    FOREX MARKETS
Name: topic, Length: 9500, dtype: object

In [4]:
#print('Features:' , count.get_feature_names())# 檢視feature names
#print('Values: \n', X.toarray())

### Defining Method
Defining different model function to be used later in the k fold validation


In [5]:
# Using specific model "method", return specifc score "score" by cross validation

def Model_Score (X, y, method, k):
    clf = method
    
    accuracy_scores = cross_val_score(clf, X, y, cv=k, scoring="accuracy")
    precision_scores = cross_val_score(clf, X, y, cv=k, scoring="precision_macro")
    recall_scores = cross_val_score(clf, X, y, cv=k, scoring="recall_macro")
    f1_scores = cross_val_score(clf, X, y, cv=k, scoring="f1_macro")
    
    return np.mean(accuracy_scores), np.mean(precision_scores), np.mean(recall_scores), np.mean(f1_scores)

#def Model_report ()
    


### Training Model (Naive Bayes)
Using K-fold validation to split the training data and validation data. Use the average score of the validation sets to evaluate the performance of the model

In [6]:
# creating 10 fold for k-fold validation
#A = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [5, 6], [7, 2], [5, 4], [7, 5], [10, 4]]) 
b = np.array([1, 2, 3, 4,5,6,7,8,9,10])

# k-fold split number 10 
k = 10

# without doing any data cleaning

bernoulliNB_accuracy, bernoulliNB_precision, bernoulliNB_recall, bernoulliNB_f1 = Model_Score(X, y, BernoulliNB(), 10)
multinomialNB_accuracy, multinomialNB_precision, multinomialNB_recall, multinomialNB_f1 = Model_Score(X, y, MultinomialNB(), 10)


# multinomialNB using uniformed distribution
multiNB_accuracy2, multiNB_precision2, multiNB_recall2, multiNB_f12 = Model_Score(X, y, MultinomialNB(fit_prior = False), 10)


# reduce irrelevant samples
R_accuracy, R_precision, R_recall, R_f1 = Model_Score(R_X, R_y, MultinomialNB(), 10)


# Icreasing the minor classes
I_accuracy, I_precision, I_recall, I_f1 = Model_Score(I_X, I_y, MultinomialNB(), 10)


# Try Classification report
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

clf_BernoulliNB = BernoulliNB()
model_BernoulliNB = clf_BernoulliNB.fit(X_train, y_train)

clf_MultinomialNB = MultinomialNB()
model_MultinomialNB = clf_MultinomialNB.fit(X_train, y_train)


predicted_BernoulliNB = model_BernoulliNB.predict(X_valid)
predicted_MultinomialNB = model_MultinomialNB.predict(X_valid)

I_X_train, I_X_valid, I_y_train, I_y_valid = train_test_split(I_X, I_y, test_size=0.20, random_state=42)
model_MultinomialNB_2 = clf_MultinomialNB.fit(I_X_train, I_y_train)
predicted_MultinomialNB_2 = model_MultinomialNB_2.predict(I_X_valid)





# still need to deal with 


# 1. irrelevant articles

# 2. The distribution of topics are not uniform

# 3. select the features

# 4. maybe can give penalty to the misclassifying




# multinomialNB_accuracy = Model_Score(X, y, MultinomialNB(), 10, "accuracy")


F:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
F:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
F:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
F:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
F:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Preci

In [7]:
print("====================================================================================")
print("Without doing any data cleaning, the score of bernoulliNB,\naccuracy:  " + str(bernoulliNB_accuracy) +
     "\nprecision: " + str(bernoulliNB_precision) + "\nrecall:    " + str(bernoulliNB_recall) + "\nf1:        " +
     str(bernoulliNB_f1))
#print("\nClassification Report for bernoulliNB:\n")
#print(classification_report(y_valid, predicted_BernoulliNB))

print("====================================================================================")

print("Without doing any data cleaning, the score of multinomialNB,\naccuracy:  " + str(multinomialNB_accuracy) +
     "\nprecision: " + str(multinomialNB_precision) + "\nrecall:    " + str(multinomialNB_recall) + "\nf1:        " +
     str(multinomialNB_f1))
#print("\nClassification Report for multinomialNB:\n")
#print(classification_report(y_valid, predicted_MultinomialNB))

print("====================================================================================")

print("Setting uniformed prior, the score of multinomialNB,\naccuracy:  " + str(multiNB_accuracy2) +
     "\nprecision: " + str(multiNB_precision2) + "\nrecall:    " + str(multiNB_recall2) + "\nf1:        " +
     str(multiNB_f12))
#print("\nClassification Report for multinomialNB:\n")
#print(classification_report(y_valid, predicted_MultinomialNB))

print("====================================================================================")

print("Reduce the case of irrelevant, the score of multinomialNB,\naccuracy:  " + str(R_accuracy) +
     "\nprecision: " + str(R_precision) + "\nrecall:    " + str(R_recall) + "\nf1:        " +
     str(R_f1))

print("====================================================================================")

print("Increase(Copy) the case of topic classes, the score of multinomialNB,\naccuracy:  " + str(I_accuracy) +
     "\nprecision: " + str(I_precision) + "\nrecall:    " + str(I_recall) + "\nf1:        " +
     str(I_f1))

print("\nClassification Report for multinomialNB after Resampling Dataset:\n")
print(classification_report(I_y_valid, predicted_MultinomialNB_2))



Without doing any data cleaning, the score of bernoulliNB,
accuracy:  0.7038942167520311
precision: 0.37212369276862634
recall:    0.2819154032170111
f1:        0.28388746010825894
Without doing any data cleaning, the score of multinomialNB,
accuracy:  0.7358161832731782
precision: 0.6306736067435463
recall:    0.5526188603489018
f1:        0.5583352913640214
Setting uniformed prior, the score of multinomialNB,
accuracy:  0.7338169401702015
precision: 0.6325643464367477
recall:    0.5709569773293275
f1:        0.5712769594923721
Reduce the case of irrelevant, the score of multinomialNB,
accuracy:  0.7106463220517194
precision: 0.7075125677990888
recall:    0.6049673605345067
f1:        0.6255152766267569
Increase(Copy) the case of topic classes, the score of multinomialNB,
accuracy:  0.8046992057147376
precision: 0.8135680332176245
recall:    0.8751822879751108
f1:        0.8346417211219871

Classification Report for multinomialNB after Resampling Dataset:

                            

### Final Test Try

In [8]:
#creating a test for test data separately would result in dimension mismatch

'''
cv = CountVectorizer()
X_t = count.fit_transform(raw_testing["article_words"])
y_t = raw_testing["topic"]

print(type(X_valid))
print(len(X_valid.toarray()[0]))
print(type(X_t))
print(len(X_t.toarray()[0]))

proba_y = model_MultinomialNB_2.predict(X_t)

<class 'scipy.sparse.csr.csr_matrix'>
35822
<class 'scipy.sparse.csr.csr_matrix'>
52

ValueError: dimension mismatch
'''

'\ncv = CountVectorizer()\nX_t = count.fit_transform(raw_testing["article_words"])\ny_t = raw_testing["topic"]\n\nprint(type(X_valid))\nprint(len(X_valid.toarray()[0]))\nprint(type(X_t))\nprint(len(X_t.toarray()[0]))\n\nproba_y = model_MultinomialNB_2.predict(X_t)\n\n<class \'scipy.sparse.csr.csr_matrix\'>\n35822\n<class \'scipy.sparse.csr.csr_matrix\'>\n52\n\nValueError: dimension mismatch\n'

In [9]:
# append raw_testing["article_words"] and raw_testing["topic"] to get consist dimension in the test
topic_class = raw_training[raw_training["topic"] != "IRRELEVANT"]
increase_training = pd.concat([topic_class, topic_class, topic_class, raw_training[raw_training["topic"] == "IRRELEVANT"]], ignore_index=True)
train_size = len(increase_training)
print(train_size)
result = increase_training["article_words"].append(raw_testing["article_words"])
print(len(result))

increase_bag_of_words = count.fit_transform(result)
I_X = increase_bag_of_words
I_y = increase_training["topic"].append(raw_testing["topic"])

print(len(I_y))


19032
19532
19532


In [10]:
for c in model_MultinomialNB_2.classes_:
    print(c)

ARTS CULTURE ENTERTAINMENT
BIOGRAPHIES PERSONALITIES PEOPLE
DEFENCE
DOMESTIC MARKETS
FOREX MARKETS
HEALTH
IRRELEVANT
MONEY MARKETS
SCIENCE AND TECHNOLOGY
SHARE LISTINGS
SPORTS


In [11]:
I_X_train = I_X[:train_size]
I_X_test = I_X[train_size:]
I_y_train = I_y[:train_size]
I_y_test = I_y[train_size:]

model_MultinomialNB_2 = clf_MultinomialNB.fit(I_X_train, I_y_train)
proba_y = model_MultinomialNB_2.predict_proba(I_X_test)
print(proba_y)
count = 0
recommendation = dict()

for c in model_MultinomialNB_2.classes_:
    recommendation[c] = dict()

for i in range(len(proba_y)):
    #print(proba_y[i])
    #print(predicted_MultinomialNB_2[i])

    proba = np.max(proba_y[i])
    pred = predicted_MultinomialNB_2[i]
    true = I_y_test.array[i]
    id = 9501+i
    #print("id ", id)
    #print("pred ", pred)
    #print("proba ", proba)
    #print("true  ", true)
    if(pred == true):
        count+=1
        recommendation[pred][id] = proba
print(count, len(proba_y), count/len(proba_y))


[[6.94627027e-034 1.48561093e-045 4.14253628e-033 ... 4.03153273e-039
  9.99999998e-001 2.79687735e-063]
 [8.41709112e-004 3.85527320e-005 2.07838284e-008 ... 1.64175632e-014
  1.57817415e-009 7.24703136e-015]
 [4.47199805e-132 1.73460607e-128 9.77617803e-134 ... 4.45934322e-168
  4.36857252e-085 8.73359063e-159]
 ...
 [1.38604807e-052 6.25993590e-050 1.22171378e-059 ... 3.14548643e-062
  4.56774797e-045 2.25874435e-064]
 [2.29436041e-026 2.99121657e-027 1.14242197e-025 ... 4.98703915e-035
  1.00000000e+000 1.55102828e-027]
 [3.38511646e-100 3.21569833e-106 5.74821293e-116 ... 2.96622785e-128
  1.06071350e-077 9.23401914e-089]]
86 500 0.172


In [12]:
import operator
for key in recommendation:
    d = recommendation[key];
    sorted_d = dict(sorted(d.items(), key=operator.itemgetter(1),reverse=True))
    print(key)
    count = 0
    for k in sorted_d:
        print(k, sorted_d[k])
        count+=1
        if count == 10:
            break

ARTS CULTURE ENTERTAINMENT
BIOGRAPHIES PERSONALITIES PEOPLE
DEFENCE
9616 1.0
DOMESTIC MARKETS
FOREX MARKETS
9525 0.9999999939433337
9551 0.9999999905656978
9659 0.9999999873775778
9529 0.9927546477636874
9708 0.9743567487101559
9733 0.8236482033513559
HEALTH
IRRELEVANT
9515 1.0
9527 1.0
9605 1.0
9624 1.0
9741 1.0
9754 1.0
9781 1.0
9790 1.0
9794 1.0
9797 1.0
MONEY MARKETS
9516 1.0
9553 1.0
9572 1.0
9586 1.0
9599 0.9999999999984084
9916 0.9999999999982379
9820 0.9999999999783427
9748 0.9999999595761445
9638 0.9999999444833365
9550 0.9999999408703406
SCIENCE AND TECHNOLOGY
SHARE LISTINGS
SPORTS
9536 1.0
9597 1.0
9630 1.0
9657 1.0
9752 1.0
9800 1.0
9801 1.0
9848 1.0
9919 1.0
9942 1.0


### Training Model (Decision Tree)
Using K-fold validation to split the training data and validation data. Use the average score of the validation sets to evaluate the performance of the model


In [32]:
decisionTree_accuracy, decisionTree_precision, decisionTree_recall, decisionTree_f1 = Model_Score(X, y, DecisionTreeClassifier(), 10)


clf_decisionTree = DecisionTreeClassifier()
model_decisionTree = clf_decisionTree.fit(X_train, y_train)

predicted_decisionTree = model_decisionTree.predict(X_valid)






In [33]:
print("\n====================================================================================\\n")
print("Without doing any data cleaning, the score of decisionTree,\naccuracy:  " + str(decisionTree_accuracy) +
     "\nprecision: " + str(decisionTree_precision) + "\nrecall:    " + str(decisionTree_recall) + "\nf1:        " +
     str(decisionTree_f1))
print("\nClassification Report for decisionTree:\n")
print(classification_report(y_valid, predicted_decisionTree))


====================================================================================\n
Without doing any data cleaning, the score of decisionTree,
accuracy:  0.6996523800897622
precision: 0.5381255307309976
recall:    0.49843501674178625
f1:        0.5162767074832546

Classification Report for decisionTree:

                                  precision    recall  f1-score   support

      ARTS CULTURE ENTERTAINMENT       0.64      0.32      0.42        22
BIOGRAPHIES PERSONALITIES PEOPLE       0.31      0.21      0.25        39
                         DEFENCE       0.56      0.45      0.50        44
                DOMESTIC MARKETS       0.60      0.56      0.58        27
                   FOREX MARKETS       0.40      0.40      0.40       174
                          HEALTH       0.47      0.35      0.40        49
                      IRRELEVANT       0.78      0.82      0.80       909
                   MONEY MARKETS       0.59      0.60      0.60       344
          SCIENCE AND 

In [9]:
collections.Counter(y)

Counter({'FOREX MARKETS': 845,
         'MONEY MARKETS': 1673,
         'SPORTS': 1102,
         'IRRELEVANT': 4734,
         'SHARE LISTINGS': 218,
         'BIOGRAPHIES PERSONALITIES PEOPLE': 167,
         'DOMESTIC MARKETS': 133,
         'DEFENCE': 258,
         'SCIENCE AND TECHNOLOGY': 70,
         'HEALTH': 183,
         'ARTS CULTURE ENTERTAINMENT': 117})